Используя схемы переменных направлений и дробных шагов, решить двумерную начально-краевую задачу для дифференциального уравнения параболического типа. В различные моменты времени вычислить погрешность численного решения путем сравнения результатов с приведенным в задании аналитическим решением  $U(x, t)$. 

Исследовать зависимость погрешности от сеточных параметров  $\tau$, $h_x$, $h_y$.

$$ \frac{\partial u}{\partial t} = a \frac{\partial^2 u}{\partial x^2} + b \frac{\partial^2 u}{\partial y^2} + \sin{x} \sin{y} [\cos{\mu t} + (a+b) \sin{\mu t}] $$

$$ u(0, y, t) = 0 \\ u_x(\pi, y, t) = -\sin{y} \sin{\mu t}  \\ u(x, 0, t) = 0 \\  u_y(x, \pi, t) = -\sin{x} \sin{\mu t} \\ u(x, y, 0) = 0

В) $a = 1, b = 2, \mu = 1$

аналитическое решение $$ U(x, y, t) = \sin{x} \sin{y} \sin{\mu t} $$

In [2]:
import math
import typing
from typing import List
import matplotlib.pyplot as plt
import copy

### Входные условия

In [3]:
x_left = 0
x_right = math.pi
y_left = 0
y_right = math.pi

a = 1
b = 2
mu = 1

In [ ]:
def phi_1(x:float, t:float, mu:float=1) -> float:
    return 0

def phi_2(x:float, t:float, mu:float=1) -> float:
    return -math.sin(x) * math.sin(mu * t)

def phi_3(y:float, t:float, mu:float=1) -> float:
    return 0

def phi_4(y:float, t:float, mu:float=1) -> float:
    return -math.sin(y) * math.sin(mu * t)

def psi(x:float, y:float, mu:float=1) -> float:
    return 0

def f(x:float, y:float, t:float, mu:float=1, a:float=1, b:float=2) -> float:
    return math.sin(x) * math.sin(y) * (mu * math.cos(mu * t) + (a + b) * math.sin(mu * t))


def U(x:float, y:float, t:float, mu:float=1) -> float:
    return math.sin(x) * math.sin(y) * math.sin(mu * t)

def real_U(X:List[float], Y:List[float], T:List[float]) -> List[List[List[float]]]:
    U_true = [[[0] * len(X) for _ in range(len(Y))] for __ in range(len(T))]
    for i in range(len(T)):
        for k in range(len(Y)):
            for j in range(len(X)):
                U_true[k][j] = U(X[j], Y[k], T[i])
    return U_true

### Вспомогательные функиции

In [ ]:
def plot_graphs(new_X:list, new_T:list, found_U:list, U_true:list, s:str='') -> None:
    plt.plot(new_X, U_true[len(new_T) // 4 ],  label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 4 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) // 2 ], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) // 2 ], label=s, linestyle='dashdot')
    plt.plot(new_X, U_true[len(new_T) - 1], label='Аналитическое решение')
    plt.plot(new_X, found_U[len(new_T) - 1],label=s, linestyle='dashdot')
    plt.legend()

In [ ]:
'''
I must fix dims HERE
'''

def local_error (U_my:list, U_true:list) -> float:
    return sum([abs(a - b) for a, b in zip(U_my, U_true)])

def get_error_array_with_h(N:list, x_left:float, x_right:float, y_left:float, y_right:float, a:float, b:float, c:float, d:float, 
                           find_u:typing.Callable, eps:float=0.1, n_x:float=10, n_y:float=10, omega:float=-10, for_x:bool=True)  -> (list, list): # H, error
    
    if for_x:
        right = x_right
    else:
        right = y_right
    H = [right/(n - 1) for n in N]
    ERROR = []
    for n in N:
        if omega == -10:
            if for_x:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d, eps=eps, n_x=n, n_y=n_y)
            else:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d, eps=eps, n_x=n_x, n_y=n)
        else:
            if for_x:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d=0, eps=eps, n_x=n, n_y=n_y, omega=omega)
            else:
                XX, YY, UU = find_u(x_left, x_right, y_left, y_right, a, b, c, d=0, eps=eps, n_x=n_x, n_y=n, omega=omega)
        U_true = real_U(XX, YY)
        t = len(YY) // 2
        ERROR.append(local_error(UU[t], U_true[t]))
    
    return H, ERROR

In [ ]:
def h_error_plot(H:list, ERROR:list, s:str=' x') -> None:
    plt.plot(H, ERROR[::-1])
    plt.xlabel("h" + s)
    plt.ylabel("error")
    plt.show()

In [ ]:
def frange(start:float, stop:float, step:float) -> float:
    while start < stop:
        yield start
        start += step

In [ ]:
def get_y(y0:float, y_end:float, h:float) -> list:
    return [i for i in frange(y0, y_end+h, h)]

def get_x(x_0:float, x_l:float, h:float) -> list:
    return [i for i in frange(x_0, x_l+h, h)]